# <center><font face="STCAIYUN" color=blue size=7>C H I S E L</font></center>

# 第3章    模块与硬件类型
---
<font face="宋体" size=3>Chisel在构建硬件的思路上与Verilog类似。在Verilog中，是以“模块(module)”为基本单位组成一个完整的独立功能实体，所以Chisel也是按模块划分的，只不过不是用关键字“module”开头来定义模块，而是用一个继承自Module类的自定义class。     
    本章将介绍Chisel里的常用硬件类型以及如何编写一个基本的模块，对于高级类型，读者可以自行研究。这些类型的语法很简单，都是由定义在单例对象里的apply工厂方法来完成。字面的名字已经把硬件含义表明得很清楚，至于它们的具体实现是什么，读者可以不用关心。
</font>
# <font face="宋体" size=3>设置</font>

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import chisel3.stage.ChiselGeneratorAnnotation

***

## Chisel是如何赋值的
<font face="宋体" size=3>有了硬件类型后，就可以用**赋值操作**来进行信号的传递或者电路的连接。只有硬件赋值才有意义，单纯的数据对象进行赋值并不会被编译器转换成实际的电路，因为在Verilog里也是对wire、reg类型的硬件进行赋值。那么，赋值操作需要什么样的操作符来完成呢？  </font>

<font face="宋体" size=3>在Chisel里，因为硬件电路的不可变性,所有对象都应该由val类型的变量来引用。Chisel类都定义了方法“**:=**”，作为等号赋值的代替。所以首次创建变量时用等号初始化，如果变量引用的对象不能立即确定状态或本身就是可变对象，则在后续更新状态时应该用“:=”。从前面讲的操作符优先级来判断，该操作符以等号结尾，而且不是四种逻辑比较符号之一，所以优先级与等号一致，是最低的。需要注意的是在Verilog中分阻塞赋值和非阻塞赋值两种赋值方式，而chisel中只有一种赋值方式。例如：
</font>

In [ ]:
val x = Wire(UInt(4.W))
val y = Wire(UInt(4.W))
x := "b1010".U  // 向4bit的线网x赋予了无符号数10
y := ~x  // 把x按位取反，传递给y

## 端口
### 定义端口列表

<font face="宋体" size=3>定义一个模块前首先需要定义好端口。整个端口列表是由方法定义为：“**IO[T <: Data](iodef: T)**”。其参数通常是一个Bundle类型的对象，而且引用的字段名称必须是“io” （对于继承自Module的模块）。因为端口存在方向，所以还需要方法“Input[T <: Data](source: T)”和“Output[T <: Data](source: T)”来为每个端口表明具体的方向。        
注意，“Input[T <: Data](source: T)”和“Output[T <: Data](source: T)”的入参是数据类型，不能是硬件类型。目前Chisel还不支持双向端口inout，只能通过黑盒里的Analog端口来模拟外部Verilog的双向端口。
</font>


In [ ]:
class MyIO extends Bundle {
   val in = Input(Vec(5, UInt(32.W)))
   val out = Output(UInt(32.W))
}
class MyModule extends Module{
   val io = IO(new MyIO)  // 模块的端口列表
}

<font face="宋体" size=3>一旦端口列表定义完成，就可以通过“io.xxx”来使用。</font>
### 翻转端口列表的方向
<font face="宋体" size=3>Chisel提供了“Flipped[T <: Data](source: T)”方法，可以把参数里所有的输入端口转为输出端口，输出端口转为输入端口。如果是黑盒里的Analog端口，则仍是双向的。</font>


In [ ]:
class MyIO extends Bundle {
   val in = Input(Vec(5, UInt(32.W)))
   val out = Output(UInt(32.W))
}
class MyModule_1 extends Module{
val io = IO(new MyIO)  // in是输入，out是输出
}
class MyModule_2 extends Module{
val io = IO(Flipped(new MyIO))  // out是输入，in是输出
}

### 整体连接
<font face="宋体" size=3>翻转方向的端口列表通常配合整体连接符号“**<>**”使用。该操作符会把左右两边的端口列表里所有同名的端口进行连接，而且同一级的端口方向必须是输入端口连接输出端口、输出端口连接输入端口，父级和子级的端口方向则是输入端口与输入端口相连、输出端口与输出端口相连</font>

In [ ]:
class MyIO extends Bundle {
   val in = Input(Vec(5, UInt(32.W)))
   val out = Output(UInt(32.W))
}
class MyModule extends Module{
val io = IO(new Bundle {
       val x = new MyIO       
       val y = Flipped(new MyIO)
   })
	io.x <> io.y  // 相当于 io.y.in := io.x.in; io.x.out := io.y.out
}

### 动态修改端口 
<font face="宋体" size=3>（1）使用可选字段 Chisel通过引入Scala的Boolean参数、可选值以及if语句可以创建出可选的端口，在例化该模块时可以通过控制Boolean入参来生成不同的端口。</font>

In [ ]:
class ModuleWithOptionalIOs(flag: Boolean) extends Module {
   		val io = IO(new Bundle {
       		val in = Input(UInt(12.W))
       		val out = Output(UInt(12.W))
       		val out2 = if (flag) Some(Output(UInt(12.W))) else None
 		 })
  
 		io.out := io.in
   		if(flag) {
     		io.out2.get := io.in
   		}
} 

<font face="宋体" size=3>（2）使用Zero-Width chisel允许数据的位宽为0。位宽为0的IO会从生成的Verilog中去除，当你试图使用位宽为0的wire时，你会得到一个常数0。如果0是一个合理的默认值，那么更推荐使用该方法。</font>

In [ ]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = Input(if (hasCarry) UInt(1.W) else UInt(0.W))
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn
  io.s := sum(0)
  io.carryOut := sum(1)
}

## 模块

### 模块分类

|模块|作用|
|:----:|:----:|
|Module|有一个隐式时钟（称为clock）和一个隐式复位（称为reset），必须实现抽象成员io|
|MultiIOModule|有一个隐式时钟和复位；Module有一个隐式时钟（称为clock）和一个隐式复位（称为reset），必须实现抽象成员io|
|RawModule|允许根据需要定义IO，无抽象成员io，但不提供隐式时钟和复位，可以自定义时钟和复位信号|

<font face="宋体" size=3>下面分别用三种模块实现与门，注意区别。</font>


In [ ]:
class AndRawModule extends RawModule {
  val ioBundle = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
  })
  val c = IO(Output(UInt(1.W)))
  c := ioBundle.a & ioBundle.b
}
class AndMultiIOModule extends MultiIOModule {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val c = Output(UInt(1.W))
  })
  val iob = IO(Input(UInt(1.W)))
  io.c := io.a & iob
}
class AndModule extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val c = Output(UInt(1.W))
  })
  io.c := io.a & io.b
}

### 定义模块
<font face="宋体" size=3>在Chisel里定义一个模块一般是通过继承Module类来实现的，这个类有以下三个特点：  
    ①继承自Module类。   
    ②包含一个用于接口的抽象字段“io”，该字段必须引用前面所说的端口对象。   
    ③在类的主构造器里进行内部电路连线。
</font>

In [ ]:
// mux2.scala
class Mux2 extends Module {
  val io = IO(new Bundle{
    val sel = Input(UInt(1.W))
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val out = Output(UInt(1.W))
  })
 
  io.out := (io.sel & io.in1) | (~io.sel & io.in0)
}


### 例化模块
<font face="宋体" size=3>要例化一个模块，并不是直接用new生成一个实例对象就完成了，还需要再把实例的对象传递给单例对象Module的apply方法。</font>

In [ ]:
// mux4.scala
class Mux4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val in2 = Input(UInt(1.W))
    val in3 = Input(UInt(1.W))
    val sel = Input(UInt(2.W))
    val out = Output(UInt(1.W))
  })
  val m0 = Module(new Mux2)
  m0.io.sel := io.sel(0)
  m0.io.in0 := io.in0
  m0.io.in1 := io.in1
  val m1 = Module(new Mux2)
  m1.io.sel := io.sel(0)
  m1.io.in0 := io.in2
  m1.io.in1 := io.in3
  val m2 = Module(new Mux2)
  m2.io.sel := io.sel(1)
  m2.io.in0 := m0.io.out
  m2.io.in1 := m1.io.out
  io.out := m2.io.out
}


### 例化多个模块
<font face="宋体" size=3>在上个例子中，模块Mux2被例化了三次，实际上只需要一次性例化三个模块就可以了。对于要多次例化的重复模块，可以利用向量的工厂方法。因为该方法接收的参数类型是Data的子类，而模块的字段io正好是Bundle类型，并且实际的电路连线仅仅只需针对模块的端口，所以可以把待例化模块的io字段组成一个序列，或者按重复参数的方式作为参数传递。
</font>

In [ ]:
// mux4_2.scala
class Mux4_2 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val in2 = Input(UInt(1.W))
    val in3 = Input(UInt(1.W))
    val sel = Input(UInt(2.W))
    val out = Output(UInt(1.W))
  })
  val m = VecInit(Seq.fill(3)(Module(new Mux2).io))  
// 例化了三个Mux2，并且参数是端口字段io
  m(0).sel := io.sel(0)  // 模块的端口通过下标索引，并且路径里没有“io”
  m(0).in0 := io.in0
  m(0).in1 := io.in1
  m(1).sel := io.sel(0)
  m(1).in0 := io.in2
  m(1).in1 := io.in3
  m(2).sel := io.sel(1)
  m(2).in0 := m(0).out
  m(2).in1 := m(1).out
  io.out := m(2).out
}


<font face="宋体" size=3>这样写和上例中分别例化三个模块生成的Verilog代码是相同的，但在Verilog中没有Chisel这样一次例化多个模块这样的语法，这体现了Chisel在编写大规模电路时的高效性。实际项目中例化多个模块可能是要带不同参数的，无法用VecInit例化，只能用for循环+yield，用循环/控制语句控制例化模块，下面我们来看一个示例。</font>

In [ ]:
// MoudleForYield.scala

//n位全加器
class NBitsAdder(n: Int) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(n.W))
    val b = Input(UInt(n.W))
    val s = Output(UInt((n+1).W))
  })
//使用“+&”运算符，结果拓展位宽
  io.s := io.a +& io.b
}

/*将n个m位宽的数据累加起来，n为2的次幂，比如n=4时，计算a+b+c+d，先用两个加法器计算a+b和c+d，再使用一个位宽更大的加法器将上述两个结果相加得到最终结果。*/
class MoudleForYield(n: Int, m: Int) extends Module {
  //输入数据的个数
  val inputNum = n
  val outputWidth = log2Up(n+1) + m - 1
  val io = IO(new Bundle{
    //输入使用Vec向量，表示n个m位宽的UInt
    val in = Input(Vec(n,UInt(m.W)))
    val out = Output(UInt(outputWidth.W))
  })
  //stage表示加法器的级数，0表示第1级，例如n=8时需要3级加法器
  var stage = 0
  //stageNum表示每级所需加法器个数，例如n=8时，每级依次为4、2、1个
  var stageNum = inputNum/2
  //使用可变映射Map保存例化的模块，key为stage，value为yield生成的序列
  val mutableMap = scala.collection.mutable.Map[Int, IndexedSeq[NBitsAdder]]()
  //使用while循环逐级例化加法器，每高一级位宽加一
  while (stage < outputWidth-m) {
    mutableMap += ( stage -> (for (i <- 1 to stageNum) yield Module(new NBitsAdder(stage+m))) )
    stage = stage + 1
    stageNum = stageNum/2
  }

  //例化完成后进行连线，与主模块连接的部分比较特殊，需要单独连线
  //num是临时变量，用于索引
  var num = 0
  //将主模块io.in与mutableMap(0)即第1级加法器输入相连
  //在for循环中将mutableMap(0)中的元素进行遍历并连线
  for (adder <- mutableMap(0)) {
    adder.io.a := io.in(num)
    adder.io.b := io.in(num+1)
    num = num + 2
  }

  //num为key，num1为索引
  num = 1
  var num1 = 0
  /*使用while循环逐级进行连线，将上一级两个加法器的输出接到这一级加法器的输入，mutableMap.getOrElse(num,-1)作用是返回key为num对应的value，找不到key为num的value则返回-1，详情见14.5.2。*/
  while (mutableMap.getOrElse(num,-1) != -1) {
    for (adder <- mutableMap(num)) {
      adder.io.a := mutableMap(num-1)(num1).io.s
      adder.io.b := mutableMap(num-1)(num1+1).io.s
      num1 = num1 + 2
    }
    num = num + 1
    num1 = 0
  }
  //最后将末级最后一个加法器的输出接到主模块的io.out端口
  io.out := mutableMap(num-1)(0).io.s
}


In [ ]:
object MoudleForYieldGen extends App {
  new (chisel3.stage.ChiselStage).execute(Array("--target-dir", "./generated/chapter03/MoudleForYield"),
    Seq(ChiselGeneratorAnnotation(() => new MoudleForYield(8,4))))
}


<font face="宋体" size=3>生成的8个4位宽的该加法器生成的verilog代码详见教材部分。可以从生成的Verilog代码中看出第1级使用了4个4位宽的加法器，第2级是两个5位宽的加法器，第3级是一个6位宽的加法器，最终结果7位宽，符合预期。</font>

## 线网
### Wire

<font face="宋体" size=3>Chisel把线网作为电路的节点，通过工厂方法“Wire[T <: Data](t: T)”来定义线网，相当于Verilog中定义的wire类型的变量。可以对线网进行赋值，也可以把线网连接到其他电路节点，线网是组成组合逻辑的基本硬件类型。</font>

In [ ]:
class MyModule extends Module{
    val io = IO(new Bundle{
        val in = Input(UInt(8.W))
})

val myNode = Wire(UInt(8.W))
myNode := 0.U;
myNode := io.in + 1.U;
}

<font face="宋体" size=3>因为Scala作为软件语言是顺序执行的，定义具有覆盖性，所以如果对同一个线网多次赋值，则只有最后一次赋值是有效的。例如下面的代码与上面的例子是等效的：
</font>

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
  })

  val myNode = Wire(UInt(8.W))
  myNode := 0.U;
  myNode := io.in + 1.U;
}


<font face="宋体" size=3>Wire的一些用例如下[1]，如果不提供位宽参数，代码将启用自动推断；而在Verilog中不支持位宽的自动推断，如不写出位宽默认为1位。</font>

In [ ]:
val w0 = Wire(UInt()) // width is inferred
val w1 = Wire(UInt(8.W)) // width is set to 8

val w2 = Wire(Vec(4, UInt())) // width is inferred
val w3 = Wire(Vec(4, UInt(8.W))) // width of each element is set to 8

class MyBundle {
  val unknown = UInt()
  val known   = UInt(8.W)
}
val w4 = Wire(new MyBundle)
// Width of w4.unknown is inferred
// Width of w4.known is set to 8


### WireDefault

### 未驱动的线网
<font face="宋体" size=3>Chisel的Invalidate API支持检测未驱动的输出型IO以及定义不完整的Wire定义，在编译成firrtl时会产生“not fully initialized”错误。换句话说，就是组合逻辑的真值表不完整，不能综合出完整的电路。如果确实需要不被驱动的线网，则可以赋给一个DontCare对象，这会告诉Firrtl编译器，该线网故意不被驱动。
</font>

## 寄存器
<font face="宋体" size=3>寄存器是时序逻辑的基本硬件类型，它们都是由当前时钟域的时钟上升沿触发的。如果模块里没有多时钟域的语句块，那么寄存器都是由隐式的全局时钟来控制。对于有复位信号的寄存器，如果不在多时钟域语句块里，则由隐式的全局复位来控制，并且复位信号是高有效。     
    chisel有五种内建的寄存器：Reg、RegNext、RegInit、util包里的RegEnable和ShiftRegister，下面逐一介绍。

</font>

|寄存器|名称|定义方法|作用|
|:----:|:----:|:----:|:----:|
|Reg|普通的寄存器|Reg[T <: Data](t: T)|它可以在when语句里用全局reset信号进行同步复位，也可以进行条件赋值或无条件跟随|
|RegNext|跟随寄存器|RegNext[T <: Data](next: T)|返回一个位宽可以自动推断的寄存器，在每个时钟上升沿，它都会采样一次传入的参数，并且没有复位信号|
|RegInit|复位到指定值的寄存器|RegInit[T <: Data](init: T)|可以用内建的when语句进行条件赋值，当前隐式复位信号有效时，寄存器将被设置为初始化值|
|RegEnable|带一个使能端的寄存器RegEnable|apply[T <: Data](next: T, enable: Bool)|不进行复位初始化|
|ShiftRegister|移位寄存器|ShiftRegister[T <: Data](in: T, n: Int, resetData: T, en: Bool)|不进行复位初始化|

#### 寄存器实例

In [ ]:
// reg.scala
class REG extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(8.W))
    val en = Input(Bool())
    val c = Output(UInt(1.W))
  })
val reg0 = RegNext(io.a)
  val reg1 = RegNext(io.a, 0.U)
  val reg2 = RegInit(0.U(8.W))
  val reg3 = Reg(UInt(8.W))
  val reg4 = Reg(UInt(8.W))
  val reg5 = RegEnable(io.a + 1.U, 0.U, io.en)
  val reg6 = RegEnable(io.a - 1.U, io.en)
  val reg7 = ShiftRegister(io.a, 3, 0.U, io.en)
  val reg8 = ShiftRegister(io.a, 3, io.en)
  
  reg2 := io.a.andR
  reg3 := io.a.orR
  when(reset.toBool) {
    reg4 := 0.U
  } .otherwise {
    reg4 := 1.U
  }
  io.c := reg0(0) & reg1(0) & reg2(0) & reg3(0) & reg4(0) & reg5(0) & reg6(0) & reg7(0) & reg8(0)
} 


### 异步寄存器
<font face="宋体" size=3>我们使用withClockAndReset来构造异步时钟和异步复位信号，也可以用withClock和withReset单独控制异步时钟或异步复位信号，代码如下：
</font>

In [ ]:
class asyncReg extends Module {
  val io = IO(new Bundle {
    val asyncClk = Input(UInt(1.W))
    val asyncRst = Input(UInt(1.W))
    val out = Output(UInt(8.W))
  })

  val asyncRegInit = withClockAndReset(io.asyncClk.asBool().asClock(),
    io.asyncRst.asBool().asAsyncReset())(RegInit(0.U(8.W)))
  asyncRegInit := asyncRegInit + 1.U
  io.out := asyncRegInit
}


### 寄存器组
<font face="宋体" size=3>上述构造寄存器的工厂方法，它们的参数可以是任何Data的子类型。如果把子类型Vec[T]作为参数传递进去，就会生成多个位宽相同、行为相同、名字前缀相同的寄存器。同样，寄存器组在Chisel代码里可以通过下标索引。例如：
</font>

In [ ]:
// reg2.scala
class REG2 extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(8.W))
    val en = Input(Bool())
    val c = Output(UInt(1.W))
  })
  val reg0 = RegNext(VecInit(io.a, io.a))
  val reg1 = RegNext(VecInit(io.a, io.a), VecInit(0.U, 0.U))
  val reg2 = RegInit(VecInit(0.U(8.W), 0.U(8.W)))
  val reg3 = Reg(Vec(2, UInt(8.W)))
  val reg4 = Reg(Vec(2, UInt(8.W)))
  val reg5 = RegEnable(VecInit(io.a + 1.U, io.a + 1.U), VecInit(0.U(8.W), 0.U(8.W)), io.en)
  val reg6 = RegEnable(VecInit(io.a - 1.U, io.a - 1.U), io.en)
  val reg7 = ShiftRegister(VecInit(io.a, io.a), 3, VecInit(0.U(8.W), 0.U(8.W)), io.en)
  val reg8 = ShiftRegister(VecInit(io.a, io.a), 3, io.en)
  
  reg2(0) := io.a.andR
  reg2(1) := io.a.andR
  reg3(0) := io.a.orR
  reg3(1) := io.a.orR
  when(reset.toBool) {
    reg4(0) := 0.U
    reg4(1) := 0.U
  } .otherwise {
    reg4(0) := 1.U
    reg4(1) := 1.U
  }
  io.c := reg0(0)(0) & reg1(0)(0) & reg2(0)(0) & reg3(0)(0) & reg4(0)(0) & reg5(0)(0) & reg6(0)(0) & reg7(0)(0) & reg8(0)(0) &reg0(1)(0) & reg1(1)(0) & reg2(1)(0) & reg3(1)(0) & reg4(1)(0) & reg5(1)(0) & reg6(1)(0) & reg7(1)(0) & reg8(1)(0)
}


## 用when给电路赋值


<font face="宋体" size=3>在Verilog里，可以使用“if...else if...else”这样的条件选择语句来方便地构建电路的逻辑。由于Scala已经占用了“if…else if…else”语法，所以相应的Chisel控制结构改成了when语句，其语法如下：
</font>

In [ ]:
// mux2_when.scala 
class Mux2 extends Module {
  val io = IO(new Bundle{
    val sel = Input(UInt(1.W))
    val in0 = Input(UInt(1.W))
    val in1 = Input(UInt(1.W))
    val out = Output(UInt(1.W))
  })
   
   when(io.sel === 1.U){
        io.out := io.in1
}.otherwise{
    io.out := io.in0
}
}

#### <font face="宋体" size=3><font face="STCAIYUN" color=red size=4>！！！注意</font>  
<font face="宋体" size=3>“.elsewhen”和“.otherwise”的开头都有一个英文句号。when语句不仅可以用于线网赋值，还可以用于寄存器赋值，但是要注意构建组合逻辑时不能缺失“.otherwise”分支，或者在when之前进行赋值。如果在when之后进行直接赋值会覆盖掉when的条件赋值，建议还是把.othwise写全，避免出错。
</font>

<font face="宋体" size=3>在Chisel中也会用到“if...else if...else”语句，要特别注意它与“when... .elsewhen... .otherwise”在语法和用法上的区别。</font>         
<font face="宋体" size=3>前者的判断条件返回的是Scala中的Boolean类型；而后者的判断条件返回硬件类型Bool，即Chisel自己的布尔类型，为UInt类的子类。在用法上前者通常用在电路生成器的编写中，可以通过判断参数的条件来选择性地执行Scala代码，生成不同的硬件结构。而后者可以生成对应的硬件，即多路选择器。我们可以通过下面的例子进一步理解两者的区别。
</font>

In [ ]:
//对4个输入数据进行排序
class Sort4(ascending: Boolean) extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })
    
  // 根据入参控制函数comp的比较功能，判断参数用if...else语句
  def comp(l: UInt, r: UInt): Bool = {
      if (ascending) {
        l < r
      } else {
        l > r
    }
  }

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

// 要描述电路用when... .otherwise语句
  when(comp(io.in0, io.in1)) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(comp(io.in2, io.in3)) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(comp(row11, row12)) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  val row31 = Wire(UInt(16.W))
  val row32 = Wire(UInt(16.W))
  when(comp(row10, row13)) {
    row31 := row10            // preserve middle 2 elements
    row32 := row13
  }.otherwise {
    row31 := row13            // swap middle two elements
    row32 := row10
  }

  when(comp(row10, row21)) {
    io.out0 := row31            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row31
  }

  when(comp(row22, row13)) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row32
  }.otherwise {
    io.out2 := row32            // swap first two elements
    io.out3 := row22
  }
}


<font face="宋体" size=3>通常，when用于给带使能信号的寄存器更新数据，组合逻辑不常用。对于有复位信号的寄存器，尽量不要在when语句里用“reset.asBool”作为复位条件，更推荐使用RegInit来声明寄存器，这样生成的Verilog会自动根据当前的时钟域来同步复位。
</font>

----
## 总结

<font face="宋体" size=3>数据类型必须配合硬件类型才能使用，它不能独立存在，因为编译器只会把硬件类型生成对应的Verilog代码。从语法规则上来讲，这两种类型也有很大的区别，编译器会对数据类型和硬件类型加以区分。
但是从Chisel编译器的角度来看，这两者就是不一样。换句话说，硬件类型就好像在数据类型上“包裹了一层外衣(英文原文用单词binding来形容)”。
有了基本的数据类型和硬件类型后，就已经可以编写绝大多数组合逻辑与时序逻辑电路。下一章将介绍Chisel库里定义的常用原语，有了这些原语就能更快速地像搭积木一样构建电路。
</font>


# <font face="STCAIYUN" color=red size=5>E   N   D</font>